In [ ]:
import ipywidgets as widgets
from glob import glob
import shutil as xu
import os, time, datetime
import subprocess
import pathlib

dp = r"C:\Users\user\Pictures\Camera Roll"

box_dir = r"O:\path\to\boxes"
boxes = glob(f"{box_dir}{os.sep}Box_*")
try:
    highest_box = max([int(x.split(os.sep)[-1].split("_")[-1]) for x in boxes])
except:
    highest_box = 0
print(highest_box)

try:
    highest_disk = max([int(x.split(os.sep)[-1].split("_")[1]) for x in glob(f"{box_dir}{os.sep}Box_{highest_box:03}{os.sep}Disk_*_Box_*")])
except:
    highest_disk = 0
print(highest_disk)

box = widgets.IntText(value=highest_box+1, description="Box")
disk = widgets.IntText(value=highest_disk+1, description="Disk")

box_button = widgets.Button(description="New Box")
box_handle = widgets.Output()

disk_button = widgets.Button(description="New Disk")
disk_handle = widgets.Output()

disk_photo_button = widgets.Button(description="disk label")
disk_photo_handle = widgets.Output()

box_photo_button = widgets.Button(description="box label")
box_photo_handle = widgets.Output()

disk_copy_button = widgets.Button(description="Disk Copy")
disk_copy_handle = widgets.Output()

def box_action(_):
    with box_handle:
        new_box_path = box_dir + os.sep + f"Box_{box.value:03}"
        print(new_box_path)
        if os.path.exists(new_box_path) == False:
            os.mkdir(new_box_path)
            
def disk_action(_):
    with disk_handle:
        new_disk_path = f"{box_dir}{os.sep}Box_{box.value:03}{os.sep}Disk_{disk.value:03}_Box_{box.value:03}"
        print(new_disk_path)
        if os.path.exists(new_disk_path) == False:
            os.mkdir(new_disk_path)
            os.mkdir(new_disk_path + os.sep + "Content")

def disk_photo_action(_):
    photos = glob(f"{dp}{os.sep}WIN_*.jpg")
    with disk_photo_handle:
        c = 1
        for photo in photos:
            new_photo_name = f"Disk_{disk.value:03}-Label_{c:02}.jpg"
            new_photo_path = f"{dp}{os.sep}{new_photo_name}"
            os.rename(photo, new_photo_path)
            xu.copy2(new_photo_path, f"{box_dir}{os.sep}Box_{box.value:03}{os.sep}Disk_{disk.value:03}_Box_{box.value:03}{os.sep}{new_photo_name}")
            c += 1
        
def box_photo_action(_):
    photos = glob(f"{dp}{os.sep}WIN_*.jpg")
    with box_photo_handle:
        c = 1
        for photo in photos:
            os.rename(photo, f"{dp}{os.sep}Box_{box.value:03}-Label_{c:02}.jpg")
            c += 1

def disk_copy_action(_):
    src_find = subprocess.Popen(["findmnt", "-l", "|", "grep", "floppy"])
    src = src_find.split("\t")[-1]
    dst = f"{box_dir}{os.sep}Box_{box.value:03}{os.sep}Disk_{disk.value:03}_Box_{box.value:03}{os.sep}contents{os.sep}"
    errors = []
    try: 
        xu.copytree(src, dst)
    except xu.Error as e:
        errors.append(e)
    error_log = f"{box_dir}{os.sep}Box_{box.value:03}{os.sep}Disk_{disk.value:03}_Box_{box.value:03}{os.sep}errors.txt"
    with open(error_log, "a") as error_file:
        for error in errors:
            error_file.write(error)
    
        

box_button.on_click(box_action)
disk_button.on_click(disk_action)
disk_photo_button.on_click(disk_photo_action)
box_photo_button.on_click(box_photo_action)

display_box = widgets.VBox([box, disk, 
                            box_button, box_handle, box_photo_button, box_photo_handle, 
                            disk_button, disk_handle, disk_photo_button, disk_photo_handle,
                            disk_copy_button, disk_copy_handle])
display_box